In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
model = "wic6"
wic_path = f"../input/wic/dwug_es/scores/models/{model}/train"
wic_path_input = "../input/wic"
path_to_gold_data_dwug_es = "../test_data_es.csv"
path_to_annotated_data_dwug_es = "../dwug_es/data"

In [ ]:
p = next(Path(wic_path_input).glob("*.data"))
data_input = pd.read_json(p)
display(data_input)

In [ ]:
mask = data_input["sentence1"] == data_input["sentence2"]
data_input = data_input[~mask]
print(data_input.shape)

In [ ]:
mask = data_input.duplicated(subset=["sentence1", "sentence2"], keep=False)
data_input = data_input[~mask]
print(data_input.shape)

In [ ]:
data_to_be_concatenated = [pd.read_json(data) for data in Path(wic_path).rglob("*.scores")]
data_scores = pd.concat(data_to_be_concatenated, ignore_index=True)
display(data_scores)

In [ ]:
gold_data_dwug_es = pd.read_csv(path_to_gold_data_dwug_es, sep="\t")
display(gold_data_dwug_es)

In [ ]:
target_words_dwug_es = gold_data_dwug_es["word"].unique().tolist()
target_words_dwug_es

In [ ]:
data_input_expanded = data_input.merge(data_scores, on="id", how="inner")
data_input_expanded.drop(columns=["pos"], inplace=True)
display(data_input_expanded)

mask = data_input_expanded["lemma"].isin(target_words_dwug_es)
data_input_expanded_filtered = data_input_expanded[mask]
data_input_expanded_filtered = data_input_expanded_filtered.drop(columns=["start1", "end1", "start2", "end2"])
display(data_input_expanded_filtered)
print(len(data_input_expanded_filtered["lemma"].unique().tolist()))


In [ ]:
mask = data_input_expanded_filtered["sentence1"] == data_input_expanded_filtered["sentence2"]
display(data_input_expanded_filtered[mask].shape)

In [ ]:
annotated_data_to_be_concatenated_dwug_es = [
    pd.read_csv(p, sep="\t")
    for p in Path(path_to_annotated_data_dwug_es).rglob("*uses.csv")
]
annotated_data_dwug_es = pd.concat(
    annotated_data_to_be_concatenated_dwug_es, ignore_index=True
)
display(annotated_data_dwug_es)

mask = annotated_data_dwug_es["lemma"].isin(target_words_dwug_es)
annotated_data_dwug_es_filtered = annotated_data_dwug_es[mask]
annotated_data_dwug_es_filtered = annotated_data_dwug_es_filtered[
    ["lemma", "grouping", "identifier", "context"]
]
# display(annotated_data_dwug_es_filtered)
annotated_data_dwug_es_filtered = annotated_data_dwug_es_filtered[
    ~annotated_data_dwug_es_filtered.duplicated(subset="context", keep=False)
]
display(annotated_data_dwug_es_filtered)


In [ ]:
merged1 = data_input_expanded_filtered.merge(
    annotated_data_dwug_es_filtered,
    left_on=["lemma", "sentence1"],
    right_on=["lemma", "context"],
    how="inner",
).rename(columns={"identifier": "identifier1"})


merged2 = merged1.merge(
    annotated_data_dwug_es_filtered,
    left_on=["lemma", "sentence2"],
    right_on=["lemma", "context"],
    how="inner",
).rename(columns={"identifier": "identifier2"})
print(merged2.shape)

result = merged2[
    [
        "id",
        "lemma",
        "sentence1",
        "sentence2",
        "score",
        "identifier1",
        "identifier2",
        "grouping_x",
    ]
]
result["score"] = result["score"].apply(lambda x: sum(float(e) for e in x) / len(x))
result.rename(columns={"grouping_x": "grouping", "lemma": "word"}, inplace=True)
display(result)

In [ ]:
rows_with_nan = result[result.isna().any(axis=1)]  # or df[df.isnull().any(axis=1)]

print(rows_with_nan)

In [ ]:
result.to_csv(f"{model}", index=False)

In [ ]:
for tw in target_words_dwug_es:
    mask = result["word"] == tw
    df = result[mask]

    print(tw, df.shape[0])
    assert df.shape[0] <= 780
    

In [ ]:
A = pd.DataFrame({"sentence1": ["bla bla A", "bla bla B", "bla bla C"], "sentence2": ["bla bla C", "bla bla A", "bla bla B"]})
B = pd.DataFrame({"context": ["bla bla B", "bla bla C", "bla bla A"], "identifier": ["b1", "c1", "a1"]})
display(A)
print()
display(B)

In [ ]:
merged1 = A.merge(B, left_on="sentence1", right_on="context", how="left")
display(merged1)